# Imports

In [107]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE, mean_absolute_error as MAE

# Data

In [55]:
data = pd.read_csv('hours_untill_house_cooling.csv')
data.head()

,unom,t_inside,c1,ro1,V,alpha,A,t_in_5_hours,t_in_10_hours,t_in_15_hours,t_in_20_hours,t_in_25_hours,t_in_30_hours,hours_untill_cooling
0,55856.0,24,1158,933,11784,2.991059,2494.384092,2,-2,0,1,2,-1,13
1,72164.0,21,939,878,5715,3.102195,1571.267005,-12,-17,-20,-17,-20,-17,2
2,36318.0,25,1237,1340,9651,2.442125,1925.093244,14,13,8,12,8,14,65
3,79756.0,21,896,1283,7974,2.356661,2314.849455,0,1,-4,0,2,3,7
4,3673.0,24,886,1119,7350,2.645827,2656.313235,-18,-19,-21,-16,-21,-19,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,52132.0,24,1149,1337,11013,2.838799,2056.451312,10,8,4,6,7,7,35
7996,57369.0,22,1229,943,8058,2.872421,2062.673993,9,8,3,7,5,7,11
7997,50555.0,23,1037,930,5028,3.037732,1701.801399,5,4,1,1,1,3,7
7998,40903.0,22,1271,1188,3693,2.562508,1882.891394,-5,-5,-12,-9,-7,-7,4


# Model training

In [109]:
# train_test_splir
X_train, X_test, y_train, y_test = train_test_split(data.drop('hours_untill_cooling', axis=1),
                                                    data['hours_untill_cooling'], 
                                                    random_state=52)

In [110]:
catboost_model = CatBoostRegressor(n_estimators=1000,
                                   loss_function='RMSE',
                                   max_depth=6, 
                                   random_state=52,
                                   verbose=100,
                                  )

In [119]:
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

catboost_model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50, use_best_model=True)

Learning rate set to 0.067463
0:	learn: 11.3255674	test: 11.7942372	best: 11.7942372 (0)	total: 2.28ms	remaining: 2.28s
100:	learn: 2.1931732	test: 2.8663426	best: 2.8663426 (100)	total: 107ms	remaining: 956ms
200:	learn: 1.4366262	test: 2.1888736	best: 2.1888736 (200)	total: 211ms	remaining: 838ms
300:	learn: 1.1391500	test: 1.9555782	best: 1.9555782 (300)	total: 311ms	remaining: 722ms
400:	learn: 0.9670420	test: 1.8386054	best: 1.8386054 (400)	total: 417ms	remaining: 622ms
500:	learn: 0.8560863	test: 1.7703893	best: 1.7701448 (499)	total: 521ms	remaining: 519ms
600:	learn: 0.7721113	test: 1.7353184	best: 1.7353184 (600)	total: 669ms	remaining: 444ms
700:	learn: 0.7052069	test: 1.7109195	best: 1.7106931 (699)	total: 777ms	remaining: 331ms
800:	learn: 0.6490749	test: 1.6919176	best: 1.6919176 (800)	total: 883ms	remaining: 219ms
900:	learn: 0.6063572	test: 1.6764590	best: 1.6762418 (892)	total: 1.01s	remaining: 111ms
999:	learn: 0.5689359	test: 1.6670250	best: 1.6670250 (999)	total: 1.1

# Test

In [120]:
y_pred = catboost_model.predict(X_test)

In [121]:
MSE(y_test, y_pred)

2.778972198812155

In [122]:
MAE(y_test, y_pred)

0.9067493449351216

# Save modek

In [123]:
catboost_model.save_model('catboost_for_house_cooling.cbm')